In [12]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import itertools
import yaml
import os
import numpy as np
from sklearn.metrics import auc

In [13]:
results_dict = {}

for path in os.listdir('./data'):
    if path != 'YPF.csv':
        buys = pd.read_csv(os.path.join('./data', path, 'buys.csv'))
        sells = pd.read_csv(os.path.join('./data', path, 'sells.csv'))
        wallet = pd.read_csv(os.path.join('./data', path, 'wallet.csv'))
        performance = pd.read_csv(os.path.join('./data', path, 'ml_results.csv'))

        results_dict[path] = {}
        results_dict[path]['buys'] = buys.shape[0]
        results_dict[path]['sells'] = sells.shape[0]

        try:
            results_dict[path]['wallet'] = wallet.tail(1).iloc[0]['wallet']
        except:
            results_dict[path]['wallet'] = 0


        threshold_down = 0.5
        threshold_up = 0.5
        
        df_predictions = performance[(performance['y_pred']<threshold_down) | (performance['y_pred']>threshold_up)].copy()
        df_predictions['y_pred'] = np.where(df_predictions['y_pred']<threshold_down, 0, df_predictions['y_pred'])
        df_predictions['y_pred'] = np.where(df_predictions['y_pred']>threshold_up, 1, df_predictions['y_pred'])
        results_dict[path]['accuracy_th_0505'] = np.round(accuracy_score(df_predictions.y_true, df_predictions.y_pred), 4)

        threshold_down = 0.85
        threshold_up = 0.15
        df_predictions = performance[(performance['y_pred']<threshold_down) | (performance['y_pred']>threshold_up)].copy()
        df_predictions['y_pred'] = np.where(df_predictions['y_pred']<threshold_down, 0, df_predictions['y_pred'])
        df_predictions['y_pred'] = np.where(df_predictions['y_pred']>threshold_up, 1, df_predictions['y_pred'])
        results_dict[path]['accuracy_th_8515'] = np.round(accuracy_score(df_predictions.y_true, df_predictions.y_pred), 4)

        
        try:
            results_dict[path]['auc_wallet'] = auc(wallet.index, wallet['wallet'])
        except:
            results_dict[path]['auc_wallet'] = 0

results = pd.DataFrame.from_dict(results_dict, orient='index')

In [15]:
results.sort_values(by='auc_wallet', ascending=False)

,buys,sells,wallet,accuracy_th_0505,accuracy_th_8515,auc_wallet
YPF_gradient_boosting_train_window_76_train_period_1_trading_strategy_strategies.sma_ml_strategy_only_one_tunning_True,37,37,114.233944,0.7079,0.6810,3750.152060
YPF_random_forest_train_window_38_train_period_1_trading_strategy_strategies.sma_ml_strategy_only_one_tunning_True,32,31,134.115063,0.7543,0.6314,3476.249415
YPF_random_forest_train_window_76_train_period_1_trading_strategy_strategies.sma_ml_strategy_only_one_tunning_True,30,30,130.420001,0.7611,0.6341,3320.605021
YPF_logistic_regression_train_window_38_train_period_1_trading_strategy_strategies.sma_ml_strategy_only_one_tunning_True,31,30,119.552616,0.7081,0.6370,3152.560891
YPF_logistic_regression_train_window_38_train_period_7_trading_strategy_strategies.sma_ml_strategy_only_one_tunning_True,32,31,114.070105,0.6052,0.5781,3132.453135
YPF_logistic_regression_train_window_38_train_period_14_trading_strategy_strategies.sma_ml_strategy_only_one_tunning_True,30,29,105.952461,0.5290,0.5042,2792.073969
YPF_gradient_boosting_train_window_76_train_period_7_trading_strategy_strategies.sma_ml_strategy_only_one_tunning_True,29,28,110.073948,0.5584,0.5805,2755.836590
YPF_gradient_boosting_train_window_114_train_period_1_trading_strategy_strategies.sma_ml_strategy_only_one_tunning_True,29,28,103.179997,0.6844,0.6549,2583.319966
YPF_random_forest_train_window_38_train_period_7_trading_strategy_strategies.sma_ml_strategy_only_one_tunning_True,25,24,120.924227,0.6243,0.5641,2519.280127
YPF_random_forest_train_window_114_train_period_1_trading_strategy_strategies.sma_ml_strategy_only_one_tunning_True,22,22,131.379999,0.7477,0.6254,2422.464993


In [4]:
np.array([[0.88, 0.12]])[:,0]

array([0.88])

In [5]:
a = np.array([[0.88, 0.12]])
b = a[:,1][0]
b

0.12